<h2> my screen scraper

<h3><hr>install dependencies

<h4>install parser - lxml

In [1]:
!pip install lxml

You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
#i'll install parser html55lib as well
!pip install html5lib


You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


<h4> install beatiful soup 4 screen scrapper

In [4]:
!pip install beautifulsoup4

You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
#insure request library already installed
!pip install request

You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


<h3><hr>Import dependencies

In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

<h3> use beautiful soup to scrap web page pulling the tbody tag

In [19]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
# prettify shows format hmtl text to verify scrapper worked correctly
#print(soup.prettify())

#find will bring in our defined tags, if find brought in a parent
#tag ie a encloseing "div" we could use soup.a to asset children tags 
#(ie here the div child anchor tag)

table = soup.find('tbody')
#print to insure we got the correct table
#print(table.prettify())

Load latitude and longitude data and make postal code the index

In [20]:
dfll = pd.read_csv('http://cocl.us/Geospatial_data')
dfll.set_index('Postal Code', inplace=True)
dfll

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


<h3> create postal data frame

In [22]:

#intailize dataframe
df = pd.DataFrame(columns=('Postal', 'Borough', 'Neighborhood','Latitude','Longitude'))
index=0
field = 1
holdpostal = ""
for table_detail in table.find_all('td'):
    #print (table_detail.text.strip('\n'))  #strip out new line character
    
    # process postal code
    if (field == 1):
        postalcode =table_detail.text.strip('\n') 
        
        field = 2
     
    #process bourough
    elif (field == 2):
        borough=table_detail.text.strip('\n')
        good = 1
        if (borough=='Not assigned'):
            good = 0 
        field = 3
        
    #process neighborhood    
    else:
        neighborhood=table_detail.text.strip('\n') 
        if (neighborhood=='Not assigned'):
            neighborhood=borough
        if (good ==1):                # process only if borough not "not assigned"
            
            if (holdpostal != postalcode): 
                # create new row in dataframe if new postal code 
                # lookup Latitude and Longitude
                latitude= dfll.at[postalcode,'Latitude']
                longitude=dfll.at[postalcode,'Longitude']
                df.loc[index] = [postalcode,borough,neighborhood,latitude,longitude]
                #print("write ",postalcode,borough,neighborhood," to index ",index) 
                index = index +1
                holdpostal = postalcode
                
            else:
                # if postal not changed, update record
                df.at[index-1,'Neighborhood'] = neighborhood + ", " + df.at[index-1,'Neighborhood']
                #print(df.at[index-1,'Neighborhood'])
        field = 1

<h2> updated dataframe with latitude and longitude

In [23]:
df

,Postal,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h2> new df shape

In [24]:
df.shape 

(103, 5)